In [2]:
import sys
import re
import undetected_chromedriver as uc 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from unittest import result
import requests
from bs4 import BeautifulSoup
import csv
from urllib.parse import urlsplit
from collections import deque
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import email
from logging import exception
from pydoc import plain
from sqlite3 import Time
import random
from requests_html import HTMLSession
import time
from time import sleep
from urllib.parse import urlparse
from requests.exceptions import Timeout
import validators
from validators import ValidationFailure
import warnings
import time
warnings.filterwarnings('ignore')

A = (
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36"
    "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36",
)
AGENT = A[random.randrange(len(A))]

def domainExtract(url):
    domain = urlparse(url)
    return domain.netloc

def isUrl(url):
    result = validators.url(url)

    if isinstance(result, ValidationFailure):
        return False

    return result

class EmailPhoneScraper:
    def getEmailFinder(self,string):
            email = ''
            emailRegEx = re.compile('\"mailto\:[0-9a-zA-Z\@\.]{1,}\"')
            m = emailRegEx.search(string)
            if m:
                email = m.group(0)[8:-1]
            return email

    def getPhone(self,string):
            phone = ''
            phoneRegEx = re.compile('\"tel\:[\(\)\-0-9\ ]{1,}\"')
            m = phoneRegEx.search(string)
            if m:
                phone = m.group(0)[5:-1]
            return phone
        
    def findNumberHelper(self,resultString , index):
        resultNumber = ''
        
        if index != -1:
            #substring from index to 30 ahead string and convert to array
            # after the array select second string index 1 to get number 
            if(len(resultString) > index+30):
                resultNumberArray = resultString[index:index+30].replace("\n"," ").split(" ")
                print(resultNumberArray)
                numberFlag = False 
                stringFlag = False
                for rn in resultNumberArray:
                    if rn.isdigit() and not stringFlag:
                        resultNumber += rn
                        numberFlag = True
                    elif numberFlag:
                        stringFlag = True
                    
            else:
                resultNumberArray = resultString[index:len(resultString)].replace("\n"," ").split(" ")
                numberFlag = False 
                stringFlag = False
                for rn in resultNumberArray:
                    if rn.isdigit() and not stringFlag:
                        resultNumber += rn
                        numberFlag = True
                    elif numberFlag:
                        stringFlag = True 
            
            
        return resultNumber 

        
        
    def findPhoneNumber(self,soupValue):
        resultString = soupValue.getText().lower()
        resultNumber = ''
        
        index = resultString.find("phonenumber")
        if resultNumber == "":
            resultNumber = self.findNumberHelper(resultString, index)
            
        index = resultString.find("telephone")
        if resultNumber == "":
            resultNumber = self.findNumberHelper(resultString, index)
        
        index = resultString.find("phone")
        if resultNumber == "":
            resultNumber = self.findNumberHelper(resultString, index)
        
        index = resultString.find("number")
        if resultNumber == "":
            resultNumber = self.findNumberHelper(resultString, index)
        
        index = resultString.find("tel")
        if resultNumber == "":
            resultNumber = self.findNumberHelper(resultString, index)
            
        index = resultString.find("contact")
        if resultNumber == "":
            resultNumber = self.findNumberHelper(resultString, index)
            
            
        return resultNumber
            
        
        
    def findEmail(self,soupValue):
        #lower the all the string values 
        resultString = soupValue.getText().lower()
        resultEmail = ''
        
        for val in resultString.strip().replace("\n"," ").replace(","," ").replace("\""," ").split(" "):
                if re.findall('\S+@\S+', val):
                    print("the email is " + val + " end email")
                    resultEmail = val

        return resultEmail
            

    def runScraper(self):
        #Multithreading for looping All links
        try:
            with ThreadPoolExecutor(max_workers=1000) as executor:
                executor.map(self.parseData,self.original_url)
        except Exception as e: return e
        # for url in self.original_url:
            # print(url)
            # self.parseData(url)
        self.outfile.close()
    
    
    
    def __init__(self):

        text = input("Type Your Query: ")
        fileName = text.replace(' ', '-')

        secondSinceEpoch = time.time()
        timeObj = time.localtime(secondSinceEpoch)
        timeStamp = str(timeObj.tm_mday) + '-' + str(timeObj.tm_mon) + '-' + str(timeObj.tm_year) + '-' + str(timeObj.tm_hour) + '-' + str(timeObj.tm_min) + '-' + str(timeObj.tm_sec)

        super().__init__()
        files = 'scrapped data/'+fileName+'-'+timeStamp+'.csv'
        with open(files,'w', newline='',encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(["Emails", "Website","Phone Number", "Facebook","Twitter","Instagram","Linkedin","Youtube"])
        self.headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) self.leWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
        
        
        blackListLinks = []

        blackListFile = open('blacklist.txt', 'r')
        lines = blackListFile.readlines()

        for line in lines:
            blackListLinks.append(line.rstrip())

        blackListFile.close()

        crPage = 0

        totalPages = int(input("Number of Google Pages to Crawl(i.e 20, 30, 40) Remember each Google Page contains 10 results: "))
        if(totalPages <= 1000):
            totalPages = totalPages
        else:
            totalPages = 100
        i = 1
        original_url = []
        # self.original_url.append('https://www.microaire.com/contact-info/')
        while i <= totalPages:
            
            URL = "https://google.com/search?q="+text+"&start={}"
            newUrl = URL.format(crPage)
            
            chrome_options = uc.ChromeOptions()
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-dev-shm-usage")
            chrome_options.add_argument("--start-maximized")
            chrome_options.add_argument('--disable-popup-blocking')
        
            driver = uc.Chrome(options=chrome_options)
            driver.get(newUrl)
            time.sleep(15)
            results = driver.find_elements(By.XPATH, "//div[@class='yuRUbf']//a")
            result1 = len(driver.find_elements(By.XPATH, "//div[@class='yuRUbf']"))
            print('Total Results : ', result1)
            res1 = []
            for each in range(len(results)):
                each1 = results[each].get_attribute('href')
                res1.append(each1)
                            
            print(res1)
            sleep(10)
            print("Crawling Page: " + str(i))
            for rst in res1:
                try:
                    convInt = str(i)
                    x = rst
                    y = rst
                    if x:
                        print(x)
                        print(y)
                        blList = 0
                        # if rst[:-1] != "/":
                        #     rst+="/"
                        urLink = rst
                        plainDomainName = domainExtract(urLink)
                        
                        print('Checking for Blacklist')
                        for blLk in blackListLinks:
                            if(plainDomainName == blLk and blList == 0):
                                blList = 1
                                print('blocked domain')
                                print(plainDomainName, blLk)

                        if(blList != 1):
                            dblDomain = 0
                            print("Checking for Duplicate")
                            for domain in original_url:
                                if(domainExtract(domain) == plainDomainName and dblDomain == 0):
                                    dblDomain = 1
                                    print("duplicate domain found")
                                    print(plainDomainName, domainExtract(domain))

                            if(dblDomain == 0):
                                print("Adding: " + urLink)
                                original_link = urLink 
                                
                                print('crawling: ' + original_link)
                                c = 0
                                while c != 1:
                                    driver.execute_script("window.open('');")
                                    # Switch to the new window
                                    break

                                driver.switch_to.window(driver.window_handles[1])
                                driver.get(original_link)
                                time.sleep(30)

                                result =[]

                                # print(phone)
                                facebook = ''
                                twitter = ''
                                linkedin = ''
                                instagram =''
                                youtube =''
                                contactus = ''
                                email =''
                                phoneNumber= ''

                                lin = driver.find_elements(By.XPATH,"//a")
                                print('Collecting DATA !')
                                try:
                                    for link in range(len(lin)):
                                        link_collection = lin[link].get_attribute('href')  

                                        if 'facebook.com' in link_collection:
                                            facebook = link_collection
                                            print('Facebook : ', facebook)

                                        if "twitter.com" in link_collection:
                                            twitter = link_collection
                                            print('Twitter : ', twitter)

                                        if "linkedin.com" in link_collection:
                                            linkedin = link_collection
                                            print('Linkedin : ', linkedin)

                                        if "instagram.com" in link_collection:
                                            instagram = link_collection
                                            print('Instagram : ', instagram)

                                        if "youtube.com" in link_collection:
                                            youtube = link_collection
                                            print('Youtube : ', youtube)

                                        link_text = lin[link].text
                                        if '+' and '(' and ')'in link_collection:
                                            contactus = link_collection
                                            print('Contact : ', contactus)

                                        if '@.' in link_text:
                                            email = link_text
                                            print('Email : ', email)

                                        if '@.' in link_collection:
                                            email = link_collection
                                            print('Email : ', email)

                                except:
                                    print("No data found !")
                                        
                                time.sleep(5)
                                print('=========================================================================================')
                                with open(files, 'a', newline='',encoding="utf-8") as fi:
                                    writer = csv.writer(fi)
                                    writer.writerow([email, original_link, contactus, facebook, twitter, instagram, linkedin, youtube])            
                                print('Going to the next website: ', original_link)

                    
                except:
                    pass
            driver.quit()
            i = i + 1
            crPage = crPage+10
            # print(self.original_url)

    def runScraper(self):
        #Multithreading for looping All links
        try:
            with ThreadPoolExecutor(max_workers=1000) as executor:
                executor.map(self.original_url)
        except Exception as e: return e
        # for url in self.original_url:
            # print(url)
            # self.parseData(url)

emailPhoneScraper = EmailPhoneScraper()
emailPhoneScraper.runScraper()


Type Your Query: Vape cartridges in california
Number of Google Pages to Crawl(i.e 20, 30, 40) Remember each Google Page contains 10 results: 250
Total Results :  14
['https://dabconnection.com/culture/best-cartridges-california/', 'https://dabconnection.com/culture/best-cartridges-california/', 'https://webcache.googleusercontent.com/search?q=cache:kblgzmfrFsQJ:https://dabconnection.com/culture/best-cartridges-california/&cd=4&hl=en&ct=clnk&gl=pk', 'https://internationalhighlife.com/thc-cartridge-price-around-the-usa/', 'https://www.tsa.gov/travel/security-screening/whatcanibring/items/electronic-cigarettes-and-vaping-devices', 'https://kindconcentrates.com/can-you-bring-a-disposable-thc-vape-pen-on-a-plane/', 'https://barbarycoastsf.com/cartridges/', 'https://www.bestdosage.com/best-thc-pens-in-california/', 'https://cannabis.net/blog/opinion/top-11-thc-cartridges-from-california-in-2020', 'https://www.binoidcbd.com/blogs/news/buy-thc-vape-carts-california', 'https://peoplescali.com/

Facebook :  https://www.facebook.com/sharer.php?u=https://www.binoidcbd.com/blogs/news/buy-thc-vape-carts-california
Twitter :  https://twitter.com/share?text=Buy%20THC%20Vape%20Carts%20In%20California&url=https://www.binoidcbd.com/blogs/news/buy-thc-vape-carts-california
Facebook :  https://www.facebook.com/BinoidCBD
Twitter :  https://twitter.com/BinoidCBD
Instagram :  https://instagram.com/BinoidCBD
Going to the next website:  https://www.binoidcbd.com/blogs/news/buy-thc-vape-carts-california
https://peoplescali.com/cannabis-vapes-and-cartridges/
https://peoplescali.com/cannabis-vapes-and-cartridges/
Checking for Blacklist
Checking for Duplicate
Adding: https://peoplescali.com/cannabis-vapes-and-cartridges/
crawling: https://peoplescali.com/cannabis-vapes-and-cartridges/
Instagram :  https://www.instagram.com/peoplescalifornia/
Twitter :  https://twitter.com/peoples_cali?lang=en
No data found !
Going to the next website:  https://peoplescali.com/cannabis-vapes-and-cartridges/
https:

Instagram :  https://www.instagram.com/hail_mary_jane/
Youtube :  https://www.youtube.com/user/HailMaryJaneVideos
Contact :  javascript:void(0)
Twitter :  https://twitter.com/hail_mary_jane
Facebook :  https://www.facebook.com/hailmaryjane
Instagram :  https://www.instagram.com/hail_mary_jane/
Youtube :  https://www.youtube.com/user/HailMaryJaneVideos
Instagram :  https://www.instagram.com/hail_mary_jane/
Facebook :  https://www.facebook.com/hailmaryjane
Twitter :  https://twitter.com/hail_mary_jane
Youtube :  https://www.youtube.com/user/HailMaryJaneVideos
No data found !
Going to the next website:  https://www.hailmaryjane.com/thc-oil-cartridges/
https://www.leafly.com/news/strains-products/420-best-vape-carts-pods-2022
https://www.leafly.com/news/strains-products/420-best-vape-carts-pods-2022
Checking for Blacklist
Checking for Duplicate
Adding: https://www.leafly.com/news/strains-products/420-best-vape-carts-pods-2022
crawling: https://www.leafly.com/news/strains-products/420-best-

Crawling Page: 3
https://www.libertyhealthsciences.com/retailer/liberty-health-sciences-mary-esther/product/dom-pen-california-citrus-vape-cartridge-1
https://www.libertyhealthsciences.com/retailer/liberty-health-sciences-mary-esther/product/dom-pen-california-citrus-vape-cartridge-1
Checking for Blacklist
Checking for Duplicate
Adding: https://www.libertyhealthsciences.com/retailer/liberty-health-sciences-mary-esther/product/dom-pen-california-citrus-vape-cartridge-1
crawling: https://www.libertyhealthsciences.com/retailer/liberty-health-sciences-mary-esther/product/dom-pen-california-citrus-vape-cartridge-1
Going to the next website:  https://www.libertyhealthsciences.com/retailer/liberty-health-sciences-mary-esther/product/dom-pen-california-citrus-vape-cartridge-1
https://fiftyshot.com/products/ev2-vape-cartridge-california-logo
https://fiftyshot.com/products/ev2-vape-cartridge-california-logo
Checking for Blacklist
Checking for Duplicate
Adding: https://fiftyshot.com/products/ev2-

Facebook :  https://www.facebook.com/dialog/share?app_id=125832154430708&display=popup&href=https%3A%2F%2Fwww.sandiegouniontribune.com%2Fnews%2Fhealth%2Fstory%2F2019-10-14%2Fas-cdc-warns-against-all-cannabis-vaping-californias-legal-market-tries-to-ease-consumer-fears
Twitter :  https://twitter.com/intent/tweet?url=https%3A%2F%2Fwww.sandiegouniontribune.com%2Fnews%2Fhealth%2Fstory%2F2019-10-14%2Fas-cdc-warns-against-all-cannabis-vaping-californias-legal-market-tries-to-ease-consumer-fears&text=As%20CDC%20warns%20against%20all%20cannabis%20vaping%2C%20California%27s%20legal%20market%20tries%20to%20ease%20consumer%20fears
Facebook :  https://www.facebook.com/dialog/share?app_id=125832154430708&display=popup&href=https%3A%2F%2Fwww.sandiegouniontribune.com%2Fnews%2Fhealth%2Fstory%2F2019-10-14%2Fas-cdc-warns-against-all-cannabis-vaping-californias-legal-market-tries-to-ease-consumer-fears
Twitter :  https://twitter.com/intent/tweet?url=https%3A%2F%2Fwww.sandiegouniontribune.com%2Fnews%2Fhea

...

Facebook :  https://www.facebook.com/sharer/sharer.php?u=https://www.opb.org/search?q=cache:fUSXIgBnRBQJ:https://www.opb.org/article/2022/12/12/supreme-court-wont-block-california-flavored-tobacco-ban/&cd=176&hl=en&ct=clnk&gl=pk
Twitter :  https://twitter.com/intent/tweet?url=https://www.opb.org/search?q=cache:fUSXIgBnRBQJ:https://www.opb.org/article/2022/12/12/supreme-court-wont-block-california-flavored-tobacco-ban/&cd=176&hl=en&ct=clnk&gl=pk&text=Supreme%20Court%20won%E2%80%99t%20block%20California%20flavored%20tobacco%20ban
No data found !
Going to the next website:  https://webcache.googleusercontent.com/search?q=cache:fUSXIgBnRBQJ:https://www.opb.org/article/2022/12/12/supreme-court-wont-block-california-flavored-tobacco-ban/&cd=176&hl=en&ct=clnk&gl=pk
https://budmanoc.com/product-category/vape-cartridges-batteries/
https://budmanoc.com/product-category/vape-cartridges-batteries/
Checking for Blacklist
Checking for Duplicate
Adding: https://budmanoc.com/product-category/vape-cart

Crawling Page: 19
https://fortune.com/2022/12/13/supreme-court-no-stop-california-from-banning-flavored-tobacco-products/
https://fortune.com/2022/12/13/supreme-court-no-stop-california-from-banning-flavored-tobacco-products/
Checking for Blacklist
Checking for Duplicate
Adding: https://fortune.com/2022/12/13/supreme-court-no-stop-california-from-banning-flavored-tobacco-products/
crawling: https://fortune.com/2022/12/13/supreme-court-no-stop-california-from-banning-flavored-tobacco-products/
https://webcache.googleusercontent.com/search?q=cache:dsKP39_yjlgJ:https://fortune.com/2022/12/13/supreme-court-no-stop-california-from-banning-flavored-tobacco-products/&cd=181&hl=en&ct=clnk&gl=pk
https://webcache.googleusercontent.com/search?q=cache:dsKP39_yjlgJ:https://fortune.com/2022/12/13/supreme-court-no-stop-california-from-banning-flavored-tobacco-products/&cd=181&hl=en&ct=clnk&gl=pk
Checking for Blacklist
Checking for Duplicate
Adding: https://webcache.googleusercontent.com/search?q=cach

Instagram :  https://www.instagram.com/gkuaofficial/
Instagram :  https://www.instagram.com/p/CmCwh1FvlA6
Instagram :  https://www.instagram.com/p/Cl69T-USi8U
Instagram :  https://www.instagram.com/p/Cl68hJTyB0G
Instagram :  https://www.instagram.com/p/Cl6aaIXOYrX
Instagram :  https://www.instagram.com/p/Cl5TJTWr2od
Instagram :  https://www.instagram.com/p/Cl5SVDpNIE6
Instagram :  https://www.instagram.com/p/Cl5SI2aLmJ-
Instagram :  https://www.instagram.com/p/Cl5R1RJLEQc
Instagram :  https://www.instagram.com/p/Cl5RJR8LLRE
Instagram :  https://www.instagram.com/p/Cl30XoKu4mS
Instagram :  https://www.instagram.com/p/Cl2LSMbp0_J
Instagram :  https://www.instagram.com/p/ClzoW2arblN
Contact :  javascript:void(0);
Going to the next website:  https://www.gkua.com/
https://webcache.googleusercontent.com/search?q=cache:3pvFRdKBgFUJ:https://www.gkua.com/&cd=185&hl=en&ct=clnk&gl=pk
https://webcache.googleusercontent.com/search?q=cache:3pvFRdKBgFUJ:https://www.gkua.com/&cd=185&hl=en&ct=clnk&gl=p

Facebook :  https://www.facebook.com/biospacecommunity
Twitter :  https://twitter.com/BioSpace
Linkedin :  https://www.linkedin.com/groups/127165
Going to the next website:  https://webcache.googleusercontent.com/search?q=cache:5N6pRc24cf8J:https://www.biospace.com/article/releases/eureka-vapor-launches-highest-potency-pure-vape-line-at-leading-california-dispensary-under-new-california-license/&cd=187&hl=en&ct=clnk&gl=pk
https://in.pinterest.com/pin/587016132678719570/
https://in.pinterest.com/pin/587016132678719570/
Checking for Blacklist
Checking for Duplicate
Adding: https://in.pinterest.com/pin/587016132678719570/
crawling: https://in.pinterest.com/pin/587016132678719570/
Going to the next website:  https://in.pinterest.com/pin/587016132678719570/
https://www.dompen.co/
https://www.dompen.co/
Checking for Blacklist
Checking for Duplicate
Adding: https://www.dompen.co/
crawling: https://www.dompen.co/
Instagram :  http://instagram.com/dompenvape
Youtube :  https://www.youtube.com/c

TimeoutException: Message: timeout: Timed out receiving message from renderer: 300.000
  (Session info: chrome=108.0.5359.99)
Stacktrace:
Backtrace:
	(No symbol) [0x0097F243]
	(No symbol) [0x00907FD1]
	(No symbol) [0x007FD04D]
	(No symbol) [0x007EFC24]
	(No symbol) [0x007EFA5C]
	(No symbol) [0x007EE7E8]
	(No symbol) [0x007EEEF7]
	(No symbol) [0x007F7F08]
	(No symbol) [0x00803455]
	(No symbol) [0x00806766]
	(No symbol) [0x007EF2A1]
	(No symbol) [0x0080328F]
	(No symbol) [0x0085C8F2]
	(No symbol) [0x00848386]
	(No symbol) [0x0082163C]
	(No symbol) [0x0082269D]
	GetHandleVerifier [0x00C19A22+2655074]
	GetHandleVerifier [0x00C0CA24+2601828]
	GetHandleVerifier [0x00A28C0A+619850]
	GetHandleVerifier [0x00A27830+614768]
	(No symbol) [0x009105FC]
	(No symbol) [0x00915968]
	(No symbol) [0x00915A55]
	(No symbol) [0x0092051B]
	BaseThreadInitThunk [0x7703FEF9+25]
	RtlGetAppContainerNamedObjectPath [0x77717BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77717B8E+238]
